In [ ]:
import psycopg

conn_string = "External Database URL" # External Database URL 即 render 的 External Database URL 完整資訊
conn = psycopg.connect(conn_string)

print("connection established") 
conn.close()

connection established


以下為程式碼的詳細說明：

1. `import psycopg`

    - 這行是匯入 Python 的 `psycopg` 模組（psycopg3），它是 Python 用來連接並操作 PostgreSQL 資料庫的官方驅動程式。
    - 透過這個模組，你可以用 Python 程式與 PostgreSQL 伺服器溝通，執行 SQL 查詢、更新資料等。
    
2. `conn_string = "postgresql://..."`

    - 這行是定義一個 **連線字串（connection string）**，用來描述如何連接到 PostgreSQL 資料庫。
    - 格式為：

        ```
        postgresql://<使用者名稱>:<密碼>@<主機名稱>/<資料庫名稱>
        ```

    - 這串字串告訴 `psycopg` 要用哪個帳號、密碼，連到哪台主機的哪個資料庫。

3. `conn = psycopg.connect(conn_string)`

    - 這行使用 `psycopg` 提供的 `connect()` 函數，透過剛剛定義的連線字串，建立一個與 PostgreSQL 資料庫的連線物件。
    - 連線成功後，`conn` 會是一個連線實例，後續可以用它來建立游標（cursor）執行 SQL 查詢、交易控制等。
    - 若連線失敗，程式會拋出錯誤。

4. `print("connection established")`

    - 這行在連線建立成功後，印出提示訊息「connection established」，表示程式已成功連上 PostgreSQL 資料庫。
    - 方便開發者確認連線狀態。

5. 總結

    這段程式碼的主要作用是：

    1. 匯入 PostgreSQL 的 Python 連接套件 `psycopg`。
    2. 利用一組完整的連線字串資訊，建立與遠端 PostgreSQL 資料庫的連線。
    3. 連線成功後，印出提示訊息，讓使用者知道連線已經建立。

這是 Python 與 PostgreSQL 互動的第一步，後續可以用 `conn` 建立游標來執行查詢、插入、更新等資料庫操作。

---

為了解決「專案內有密碼，不想上傳至 github，不想放在程式碼內」的問題，使用其中一種解決方法：**使用環境變數（Environment Variables）**

在專案根目錄建立 `.env` 檔案，內容以「鍵=值」格式寫入環境變數，即：

```
RENDER_DATABASE=postgresql://XXXXXXXXXX
```


在程式碼中載入 `.env` 檔並讀取環境變數（即以下程式碼 ↓ ）

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # 讀取 .env 檔案中的環境變數
conn_string = os.getenv('RENDER_DATABASE')  # 取得環境變數

取得環境變數後，執行以下程式碼（兩種程式碼皆可）

In [ ]:
import psycopg
conn = psycopg.connect(conn_string)
print("connection established")
conn.close()

connection established


In [31]:
import psycopg2
conn = psycopg2.connect(conn_string)
print("connection established")
conn.close()

connection established


---

將以上程式碼做以下調整：

1. 引入 `with as` 的流程控制，使程式執行完畢後會自動關閉檔案。保護資源，避免出錯。

2. 加入處理異常的功能（ `try` 和 `except` ），這可以在代碼中檢測和處理可能引發異常的情況，以確保程序在遇到問題時不會崩潰，能夠優雅地處理錯誤情況。

In [2]:
import psycopg2
conn = psycopg2.connect(conn_string)

try:
    with psycopg2.connect(conn_string) as conn:
        print("連線成功")
except Exception as e:
    print("連線或查詢發生錯誤：", e)

連線成功


連線成功後，要取得資料庫的資料。需使用 `cursor`（游標）執行 SQL 查詢。

In [35]:
import psycopg2
# 匯入 psycopg2 套件，這是 Python 用來連接 PostgreSQL 資料庫的標準驅動程式。

conn = psycopg2.connect(conn_string)
# 使用 conn_string 建立一個資料庫連線物件 conn。
# 這行會立即嘗試連線到 PostgreSQL 伺服器，若連線失敗會拋出例外。
# 但此行建立的 conn 物件在後續程式中未使用，可視為多餘。

try:
    # 開始一個例外處理區塊，用來捕捉連線或查詢過程中可能發生的錯誤。

    with psycopg2.connect(conn_string) as conn:
        # 使用 with 語法建立資料庫連線 conn，確保連線結束後會自動關閉（釋放資源）。
        # conn 是連線物件，可以用來建立游標並執行 SQL。

        with conn.cursor() as cur:
            # 使用 with 語法建立游標 cur，游標用來執行 SQL 指令。
            # with 結束後游標會自動關閉。

            print("connection established")
            # 印出提示訊息，表示連線成功並已取得游標。

            cur.execute("SELECT * FROM 最新訊息 LIMIT 10;")
            # 執行 SQL 查詢，從資料表「最新訊息」取出前 10 筆資料。

            rows = cur.fetchall()
            # 取得查詢結果的所有資料列，rows 是一個 list，每個元素是資料列的 tuple。

            for row in rows:
                print(row)
            # 逐筆印出查詢結果的每一列資料。

except Exception as e:
    # 若 try 區塊中發生任何錯誤（連線失敗、SQL 語法錯誤等），會跳到這裡。

    print("連線或查詢發生錯誤：", e)
    # 印出錯誤訊息，方便除錯與追蹤問題。


connection established
(2, '[協助宣傳]114年公務人員特種考試原住民族考試，預定報名日期為 114年5月28日至6月6日止，預定考試日期為同年9月6日至 9月7日。', datetime.date(2023, 5, 8), '公務人員特種考試原住民族考試規則規定，自 110年1月1日起，報名原民特考應取得原住民族語言能力認 證初級以上合格證書；自113年1月1日起，報名原民特考 一、二、三等考試應取得原住民族語言能力認證中級以上 合格證書')
(1, '\t\r\n【公告】本學院因改建工程，自111年2月1日起至改建工程竣工期間，暫停受理停車使用申請。', datetime.date(2022, 1, 19), '公告本學院因原機工大樓及學員宿舍用地改建及先期樹木移植、變電站遷移等工程，考量院區人員及公共安全，自111年2月1日起至改建工程竣工期間，暫停受理學員、外聘與監評人員及月租車停車使用申請。')
(3, '[協助宣傳]TPASS 2.0公共運輸常客優惠回饋。', datetime.date(2025, 5, 9), 'TPASS 2.0常客優惠，月月領優惠回饋金，詳細訊息請上交通部公路局官網查詢。')
(4, '[協助宣導]臺北市老人健康檢查福利政策', datetime.date(2025, 4, 15), '114年臺北市老人健康檢查尚有名額！尚未接受檢查之長者請踴躍參加，各醫院預約概況詳見衛生局官網或撥打1999轉分機1807或1805，另115年老人健檢名額將自1月起開放預約!')
(5, '[協助宣傳]114年基北北桃「我的減碳存摺」', datetime.date(2025, 1, 2), '為攜手市民多加使用公共運輸，以朝淨零碳排目標前進，由政府、企業一起帶動全民綠運輸減碳，113年由基北北桃四市交通單位、公共運輸業者及悠遊卡股份有限公司共同推動，114年度擴大辦理並增加兌獎制活動，民眾累積達一 定減碳量即可登錄兌換獎項，提高民眾參與誘因，活動辦 理期間114年1月1日至12月31日止，詳細內容可至本活動官 網查閱114年活動資訊（https://co2-passbook.com/）。')


In [5]:
import os
# 匯入 Python 內建的 os 模組，用來操作系統環境變數與檔案路徑等功能。

import psycopg2
# 匯入 psycopg2 套件，Python 用來連接與操作 PostgreSQL 資料庫的驅動程式。

from dotenv import load_dotenv
# 從 python-dotenv 套件匯入 load_dotenv 函數，用來讀取 .env 檔案並將變數載入環境變數。

# 載入 .env 檔案
load_dotenv()
# 讀取專案根目錄（或指定路徑）的 .env 檔案，將裡面的設定（例如 RENDER_DATABASE）加入系統環境變數中。
# 這樣後續就可以使用 os.getenv() 取得這些設定值。

# 從環境變數取得連線字串
conn_string = os.getenv('RENDER_DATABASE')
# 從系統環境變數中取得名為 RENDER_DATABASE 的值，這通常是 PostgreSQL 的連線字串（URI 格式）。
# 透過這種方式，可以避免將敏感資訊直接寫在程式碼裡。

try:
    # 建立資料庫連線
    with psycopg2.connect(conn_string) as conn:
        # 使用 with 語法建立資料庫連線 conn，確保程式執行完或發生錯誤時會自動關閉連線，避免資源洩漏。

        # 建立游標物件
        with conn.cursor() as cur:
            # 使用 with 語法建立游標 cur，游標是用來執行 SQL 語句的物件，
            # 並且確保游標使用完後會自動關閉。

            print("成功連線至資料庫！")
            # 印出提示訊息，表示資料庫連線成功。

            # 執行 SQL 查詢
            cur.execute("SELECT * FROM 最新訊息 ORDER BY 編號 ASC;")
            # 執行一條 SQL 查詢語句，從資料表 最新訊息 中取出資料，並依照 編號 欄位排序。

            rows = cur.fetchall()
            # 取得查詢結果的所有資料列，rows 是一個 list，每個元素是資料列的 tuple。

            # 顯示查詢結果
            for row in rows:
                print(row)
            # 逐筆印出查詢結果的每一列資料。

except psycopg2.Error as e:
    # 捕捉 psycopg2 相關的資料庫錯誤，例如連線失敗、SQL 語法錯誤等。
    print(f"資料庫操作發生錯誤: {e}")
    # 印出錯誤訊息，方便除錯。

except Exception as e:
    # 捕捉其他非預期的錯誤，確保程式不會因未處理錯誤而崩潰。
    print(f"其他錯誤: {e}")
    # 印出錯誤訊息。


成功連線至資料庫！
(1, '\t\r\n【公告】本學院因改建工程，自111年2月1日起至改建工程竣工期間，暫停受理停車使用申請。', datetime.date(2022, 1, 19), '公告本學院因原機工大樓及學員宿舍用地改建及先期樹木移植、變電站遷移等工程，考量院區人員及公共安全，自111年2月1日起至改建工程竣工期間，暫停受理學員、外聘與監評人員及月租車停車使用申請。')
(2, '[協助宣傳]114年公務人員特種考試原住民族考試，預定報名日期為 114年5月28日至6月6日止，預定考試日期為同年9月6日至 9月7日。', datetime.date(2023, 5, 8), '公務人員特種考試原住民族考試規則規定，自 110年1月1日起，報名原民特考應取得原住民族語言能力認 證初級以上合格證書；自113年1月1日起，報名原民特考 一、二、三等考試應取得原住民族語言能力認證中級以上 合格證書')
(3, '[協助宣傳]TPASS 2.0公共運輸常客優惠回饋。', datetime.date(2025, 5, 9), 'TPASS 2.0常客優惠，月月領優惠回饋金，詳細訊息請上交通部公路局官網查詢。')
(4, '[協助宣導]臺北市老人健康檢查福利政策', datetime.date(2025, 4, 15), '114年臺北市老人健康檢查尚有名額！尚未接受檢查之長者請踴躍參加，各醫院預約概況詳見衛生局官網或撥打1999轉分機1807或1805，另115年老人健檢名額將自1月起開放預約!')
(5, '[協助宣傳]114年基北北桃「我的減碳存摺」', datetime.date(2025, 1, 2), '為攜手市民多加使用公共運輸，以朝淨零碳排目標前進，由政府、企業一起帶動全民綠運輸減碳，113年由基北北桃四市交通單位、公共運輸業者及悠遊卡股份有限公司共同推動，114年度擴大辦理並增加兌獎制活動，民眾累積達一 定減碳量即可登錄兌換獎項，提高民眾參與誘因，活動辦 理期間114年1月1日至12月31日止，詳細內容可至本活動官 網查閱114年活動資訊（https://co2-passbook.com/）。')


In [4]:
import os
import psycopg2
from psycopg2 import OperationalError, ProgrammingError, DatabaseError, InterfaceError, IntegrityError, Error
from dotenv import load_dotenv

# 載入 .env 檔案
load_dotenv()

# 從環境變數取得連線字串
conn_string = os.getenv("DATABASE_URL")

try:
    # 建立資料庫連線（不建立游標）
    with psycopg2.connect(conn_string) as conn:
        print("成功建立資料庫連線！")
        # 此處可直接使用 conn 進行其他操作，但不建立游標

except OperationalError as e:
    print(f"連線錯誤或資料庫伺服器問題: {e}")
except ProgrammingError as e:
    print(f"SQL語法錯誤或資料表/欄位不存在: {e}")
except IntegrityError as e:
    print(f"資料完整性違反（主鍵重複、外鍵錯誤等）: {e}")
except InterfaceError as e:
    print(f"連線中斷或游標錯誤: {e}")
except DatabaseError as e:
    print(f"資料庫錯誤（較嚴重問題）: {e}")
except Error as e:
    print(f"psycopg2 資料庫錯誤: {e}")
except Exception as e:
    print(f"其他錯誤: {e}")

連線錯誤或資料庫伺服器問題: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



強制拋出錯誤，測試錯誤處理流程

In [3]:
@app.route("/news")
def news():
    try:
        # 建立資料庫連線（不建立游標/不執行查詢）
        with psycopg2.connect(conn_string) as conn:
            # 強制拋出錯誤，測試錯誤處理流程
            raise Exception("出現錯誤")

            print("✅ 資料庫連線成功！")
    except OperationalError as e:
        print(f"🚨 連線失敗：伺服器未啟動、網路問題或參數錯誤 | 詳細訊息：{e}")
        return render_template("error.html.jinja2"), 500
    except InterfaceError as e:
        print(f"🚨 連線中斷：連線被意外關閉 | 詳細訊息：{e}")
        return render_template("error.html.jinja2"), 500
    except Error as e:
        print(f"🚨 資料庫錯誤：{e}")
        return render_template("error.html.jinja2"), 500
    except Exception as e:
        print(f"🚨 其他錯誤：{e}")
        return render_template("error.html.jinja2"), 500

    return render_template("news.html.jinja2")


NameError: name 'app' is not defined